In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from utils.plot import plot_barv, colors

theor_epses = [1.0, 2.0, 4.0, 10.0]

img_dir = 'exp_data/images/'
os.makedirs(img_dir, exist_ok=True)

exp_folder = 'exp_data/'

## Train from scratch

### 1. Average-case vs Worst-case $\theta_0$

In [ ]:
# impact of initialization
records = []
for data_name in ['mnist', 'cifar10']:
    for init_type in ['fixed_average', 'fixed_worst']:
        for epsilon in [1.0, 2.0, 4.0, 10.0]:
            emp_epses = np.array([np.load(f'{exp_folder}/model_init/{init_type}/seed{seed}/{data_name}_half_cnn_eps{epsilon}/emp_eps_loss.npy')[0] for seed in range(5)])
            test_accs = np.load(f'{exp_folder}/model_init/{init_type}/seed0/{data_name}_half_cnn_eps{epsilon}/test_set_accs.npy')

            records.append({
                'data_name': data_name,
                'init_type': init_type,
                'theor_eps': epsilon,
                'emp_eps_mean': emp_epses.mean(),
                'emp_eps_std': emp_epses.std(),
                'emp_eps_max': emp_epses.max(),
                'test_acc': test_accs.mean()
            })
results = pd.DataFrame.from_records(records)
results

In [ ]:
# impact of initialization
fig, axs = plt.subplots(1, 2)
for ax, data_name in zip(axs.flat, ['mnist', 'cifar10']):
    resultss = [results[(results['data_name'] == data_name) & (results['init_type'] == init_type)] for init_type in ['fixed_average', 'fixed_worst']]

    plot_barv(ax, resultss, ['Average-case $\\theta_0$', 'Worst-case $\\theta_0$'], title=data_name.upper())

fig.set_size_inches(12, 3)
h,l = ax.get_legend_handles_labels()
fig.legend(h, l, loc='upper center', ncol=4, bbox_to_anchor=(0.5, 1.15))
fig.savefig(f'{img_dir}/model_init.pdf', bbox_inches='tight')

### 2. Impact of pre-training epochs in worst-case $\theta_0$

In [ ]:
# compile results into pandas dataframe
n_epochss = [0, 1, 2, 3, 4, 5]
results = []
for n_epochs in n_epochss:
    grad_norms, emp_epses = [], []
    for seed in range(5):
        folder = f'{exp_folder}/pretrain_epochs/{n_epochs}epochs/seed{seed}/mnist_half_cnn_eps10.0'

        grad_norm = np.load(f'{folder}/all_grad_norms_out.npy', allow_pickle=True)[0, 0]['after'].mean()
        test_acc = np.load(f'{folder}/test_set_accs.npy').mean()
        emp_eps = np.load(f'{folder}/emp_eps_loss.npy')

        grad_norms.append(grad_norm)
        emp_epses.append(emp_eps)
    
    results.append({
        'n_epochs': n_epochs,
        'test_acc': test_acc,
        'grad_norm': np.mean(grad_norms),
        'emp_eps_mean': np.mean(emp_epses),
        'emp_eps_std': np.std(emp_epses)
    })

results = pd.DataFrame.from_records(results)
print(results)

fig, ax = plt.subplots()

# grad_norm vs n_iters
ax.plot(results['n_epochs'].to_numpy(), results['grad_norm'].to_numpy(), marker='*', color=colors[0], label='Gradient norm')
ax.set_ylabel('Average gradient norm', color=colors[0])
ax.set_xticks(n_epochss)
ax.set_xlabel('Number of pre-training epochs')

ax2 = ax.twinx()
ax2.errorbar(results['n_epochs'].to_numpy(), results['emp_eps_mean'].to_numpy(), marker='*', color=colors[1], yerr=results['emp_eps_std'].to_numpy(), capsize=3, label='$\\varepsilon_{emp}$')
ax2.set_ylabel('Empirical $\\varepsilon_{emp}$', color=colors[1])
ax2.set_ylim(0, 10)
ax2.set_xticks(n_epochss)
ax2.set_xlabel('Number of pre-training epochs')

# combined legend
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines + lines2, labels + labels2, loc=0)

fig.set_size_inches(5, 4)
fig.savefig(f'{img_dir}/grad_norms.pdf', bbox_inches='tight')

### 3. Impact of dataset size

In [ ]:
# impact of dataset size
records = []
for data_name in ['mnist', 'cifar10']:
    for n_samples in [100, 1000, -1]:
        for epsilon in [1.0, 2.0, 4.0, 10.0]:
            emp_epses = np.array([np.load(f'{exp_folder}/dataset_size/{n_samples}samples/seed{seed}/{data_name}_half_cnn_eps{epsilon}/emp_eps_loss.npy')[0] for seed in range(5)])
            test_accs = np.load(f'{exp_folder}/dataset_size/{n_samples}samples/seed0/{data_name}_half_cnn_eps{epsilon}/test_set_accs.npy')

            records.append({
                'data_name': data_name,
                'n_samples': n_samples,
                'theor_eps': epsilon,
                'emp_eps_mean': emp_epses.mean(),
                'emp_eps_std': emp_epses.std(),
                'emp_eps_max': emp_epses.max(),
                'test_acc': test_accs.mean()
            })
results = pd.DataFrame.from_records(records)
results

In [ ]:
# impact of dataset size
fig, axs = plt.subplots(1, 2)
for ax, data_name in zip(axs.flat, ['mnist', 'cifar10']):
    resultss = [results[(results['data_name'] == data_name) & (results['n_samples'] == n_samples)] for n_samples in [100, 1000, -1]]

    plot_barv(ax, resultss, ['$n = 100$', '$n = 1{,}000$', '$n = |\\mathcal{D}|$'], title=data_name.upper())

h,l = ax.get_legend_handles_labels()
fig.legend(h, l, loc='upper center', ncol=5, bbox_to_anchor=(0.5, 1.15))
fig.set_size_inches(12, 3)
fig.savefig(f'{img_dir}/dataset_size.pdf', bbox_inches='tight')

### 4. Impact of gradient clipping norm

In [ ]:
# impact of gradient clipping norm
records = []
for data_name in ['mnist', 'cifar10']:
    for max_grad_norm in [0.1, 1.0, 10.0]:
        for epsilon in [1.0, 2.0, 4.0, 10.0]:
            emp_epses = np.array([np.load(f'{exp_folder}/max_grad_norm/{max_grad_norm}/seed{seed}/{data_name}_half_cnn_eps{epsilon}/emp_eps_loss.npy')[0] for seed in range(5)])
            test_set_accs = np.load(f'{exp_folder}/max_grad_norm/{max_grad_norm}/seed0/{data_name}_half_cnn_eps{epsilon}/test_set_accs.npy')

            records.append({
                'data_name': data_name,
                'max_grad_norm': max_grad_norm,
                'theor_eps': epsilon,
                'emp_eps_mean': emp_epses.mean(),
                'emp_eps_std': emp_epses.std(),
                'emp_eps_max': emp_epses.max(),
                'test_acc_mean': test_set_accs.mean()
            })
results = pd.DataFrame.from_records(records)
results

In [ ]:
# sensitivity to grad norm
fig, axs = plt.subplots(1, 2)
for ax, data_name in zip(axs.flat, ['mnist', 'cifar10']):
    resultss = [results[(results['data_name'] == data_name) & (results['max_grad_norm'] == max_grad_norm)] for max_grad_norm in [0.1, 1.0, 10.0]]

    plot_barv(ax, resultss, ['$C = 0.1$', '$C = 1.0$', '$C = 10.0$'], title=data_name.upper())

h,l = ax.get_legend_handles_labels()
fig.legend(h, l, loc='upper center', ncol=5, bbox_to_anchor=(0.5, 1.15))
fig.set_size_inches(12, 3)
fig.savefig(f'{img_dir}/max_grad_norm.pdf', bbox_inches='tight')

## Fine-tuning last layer

### 5. Average-case vs Worst-case

In [ ]:
# impact of initialization
records = []
for init_type in ['fixed_average', 'fixed_worst']:
    for epsilon in [1.0, 2.0, 4.0, 10.0]:
        emp_epses = np.array([np.load(f'{exp_folder}/model_init_finetune/{init_type}/seed{seed}/cifar10_half_finetune_last_lr_eps{epsilon}/emp_eps_loss.npy')[0] for seed in range(5)])
        test_accs = np.load(f'{exp_folder}/model_init_finetune/{init_type}/seed0/cifar10_half_finetune_last_lr_eps{epsilon}/test_set_accs.npy')

        records.append({
            'init_type': init_type,
            'theor_eps': epsilon,
            'emp_eps_mean': emp_epses.mean(),
            'emp_eps_std': emp_epses.std(),
            'emp_eps_max': emp_epses.max(),
            'test_acc': test_accs.mean()
        })
results = pd.DataFrame.from_records(records)
results

In [ ]:
# impact of model initialisation
fig, ax = plt.subplots()

# prepare pandas dataframe for plot
resultss = [results[results['init_type'] == init_type] for init_type in ['fixed_average', 'fixed_worst']]

plot_barv(ax, resultss, ['Average-case $\\theta_0$', 'Worst-case $\\theta_0$'])

fig.set_size_inches(12, 3)
h,l = ax.get_legend_handles_labels()
fig.legend(h, l, loc='upper center', ncol=4, bbox_to_anchor=(0.5, 1.05))
fig.savefig(f'{img_dir}/model_init_finetune.pdf', bbox_inches='tight')

### 6. Impact of dataset size

In [ ]:
# impact of dataset size
records = []
for n_samples in [100, 1000, -1]:
    for epsilon in [1.0, 2.0, 4.0, 10.0]:
        emp_epses = np.array([np.load(f'{exp_folder}/dataset_size_finetune/{n_samples}samples/seed{seed}/cifar10_half_finetune_last_lr_eps{epsilon}/emp_eps_loss.npy')[0] for seed in range(5)])

        records.append({
            'n_samples': n_samples,
            'theor_eps': epsilon,
            'emp_eps_mean': emp_epses.mean(),
            'emp_eps_std': emp_epses.std(),
            'emp_eps_max': emp_epses.max()
        })
results = pd.DataFrame.from_records(records)
results

In [ ]:
# impact of samples
fig, ax = plt.subplots()

# prepare pandas dataframe for plot
resultss = [results[results['n_samples'] == n_samples] for n_samples in [100, 1000, -1]]
plot_barv(ax, resultss, ['$n = 100$', '$n = 1{,}000$', '$n = |\\mathcal{D}|$'])

fig.set_size_inches(12, 3)
h,l = ax.get_legend_handles_labels()
fig.legend(h, l, loc='upper center', ncol=5, bbox_to_anchor=(0.5, 1.05))
fig.savefig(f'{img_dir}/dataset_size_finetune.pdf', bbox_inches='tight')

### 7. Impact of gradient clipping norm

In [ ]:
# impact of gradient clipping norm
records = []
for max_grad_norm in [0.1, 1.0, 10.0]:
    for epsilon in [1.0, 2.0, 4.0, 10.0]:
        emp_epses = np.array([np.load(f'{exp_folder}/max_grad_norm_finetune/{max_grad_norm}/seed{seed}/cifar10_half_finetune_last_lr_eps{epsilon}/emp_eps_loss.npy')[0] for seed in range(5)])

        records.append({
            'max_grad_norm': max_grad_norm,
            'theor_eps': epsilon,
            'emp_eps_mean': emp_epses.mean(),
            'emp_eps_std': emp_epses.std(),
            'emp_eps_max': emp_epses.max()
        })
results = pd.DataFrame.from_records(records)
results

In [ ]:
# impact of gradient clipping norm
fig, ax = plt.subplots()

# prepare pandas dataframe for plot
resultss = [results[results['max_grad_norm'] == max_grad_norm] for max_grad_norm in [0.1, 1.0, 10.0]]
plot_barv(ax, resultss, ['$C = 0.1$', '$C = 1.0$', '$C = 10.0$'])

fig.set_size_inches(12, 3)
h,l = ax.get_legend_handles_labels()
fig.legend(h, l, loc='upper center', ncol=5, bbox_to_anchor=(0.5, 1.05))
fig.savefig(f'{img_dir}/max_grad_norm_finetune.pdf', bbox_inches='tight')